In [1]:
import pandas as pd 
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import glob

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

base_dir = os.path.dirname(os.getcwd())

# Reformat data -- wide format

In [2]:
data_reformatted = pd.DataFrame()
data_reformatted = data_reformatted.fillna(0)
counter = 0

file = os.path.join(base_dir, 'Data', 'long_gap_ratings.csv')
data = pd.read_csv(file)

features = ['awkward', 'connected', 'topics', 'laughter', 'laughter_who', 'laughter_genuine', 
            'gestures', 'gestures_describe', 'rater_know', 'rater_know_how', 'notes']
    
for feature in features:
    
    for row in range(len(data)):

        column_list = [col for col in data.columns if feature in col]

        for column in column_list:

            video = column.split(feature)[1].split('_')[1]
            
            if video.isdigit():

                data_reformatted.at[counter, 'video_num'] = video

                value = data[column][row]

                if feature in ['gestures_describe', 'notes']:
                    data_reformatted.at[counter, feature + '_' + str(row)] = str(value)
                else:
                    data_reformatted.at[counter, feature + '_' + str(row)] = value

                counter += 1
            
        counter = 0


In [3]:
data_reformatted.head()

,video_num,awkward_0,awkward_1,awkward_2,connected_0,connected_1,connected_2,topics_0,topics_1,topics_2,laughter_0,laughter_1,laughter_2,laughter_who_0,laughter_who_1,laughter_who_2,laughter_genuine_0,laughter_genuine_1,laughter_genuine_2,gestures_0,gestures_1,gestures_2,gestures_describe_0,gestures_describe_1,gestures_describe_2,rater_know_0,rater_know_1,rater_know_2,rater_know_how_0,rater_know_how_1,rater_know_how_2,notes_0,notes_1,notes_2
0,1,68.0,20.0,61.0,14.0,63.0,55.0,50.0,24.0,68.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,nan,Each person nodded slightly,nan,1.0,1.0,1.0,Both are in the same sorority as me; the perso...,The person on the left was my freshman trip le...,friends,nan,nan,nan
1,2,0.0,16.0,17.0,0.0,72.0,45.0,80.0,13.0,29.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,nan,nan,nan,2.0,1.0,2.0,NaN,The person on the left is a friend,NaN,nan,nan,nan
2,3,9.0,6.0,2.0,70.0,88.0,71.0,94.0,78.0,94.0,1.0,1.0,1.0,3.0,3.0,3.0,6.0,7.0,7.0,2.0,2.0,2.0,nan,nan,nan,2.0,2.0,1.0,NaN,NaN,classmate,nan,nan,nan
3,4,13.0,68.0,55.0,70.0,29.0,41.0,78.0,71.0,80.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,Person on the left nodding his head,The person on the left nodded their head a lot...,nan,2.0,2.0,2.0,NaN,NaN,NaN,nan,I think that they were talking about skiing at...,nan
4,5,10.0,1.0,22.0,57.0,66.0,61.0,100.0,100.0,92.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,nan,The person on the right scrunched their face a...,squeezing eyes,2.0,2.0,2.0,NaN,NaN,NaN,nan,nan,nan


# Reformat data -- long format

In [4]:
data_long = pd.wide_to_long(data_reformatted, ['awkward', 'connected',
                                               'topics', 'laughter_genuine',],
                                    i='video_num', j='rater', sep='_', suffix='\w+')

data_long = data_long.reset_index()
data_long = data_long[['video_num', 'rater', 'awkward', 'connected', 'topics', 'laughter_genuine']]


In [5]:
data_long.head()

,video_num,rater,awkward,connected,topics,laughter_genuine
0,1,0,68.0,14.0,50.0,NaN
1,2,0,0.0,0.0,80.0,NaN
2,3,0,9.0,70.0,94.0,6.0
3,4,0,13.0,70.0,78.0,NaN
4,5,0,10.0,57.0,100.0,NaN


# Compute IRR for each metric

From pre-registration: "We will compute inter-rater reliability for each question. We will use Cohen's Kappa for categorical questions. As we have more than 2 raters, we will compute all pairwise Cohen's Kappa and then average them for an overall value. We will compute Intraclass correlation coefficients for continuous questions."

**Cohen's Kappa**

interpretation of scores here: Viera, A. J., & Garrett, J. M. (2005). Understanding interobserver agreement: the kappa statistic. Fam med, 37(5), 360-363.

package: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html

In [6]:
from sklearn.metrics import cohen_kappa_score

cohen_kappas = pd.DataFrame()
cohen_kappas = cohen_kappas.fillna(0)

features_discreet_no_nans = ['laughter', 'gestures']

for feature_discreet in features_discreet_no_nans:
    
    counter = 0

    rater_0 = data_reformatted[feature_discreet + '_0']
    rater_1 = data_reformatted[feature_discreet + '_1']
    
    cohen_kappas.at[counter, 'rater_A'] = 0
    cohen_kappas.at[counter, 'rater_B'] = 1
    cohen_kappas.at[counter, feature_discreet] = cohen_kappa_score(rater_0, rater_1)
    
    counter += 1
    
    rater_0 = data_reformatted[feature_discreet + '_0']
    rater_2 = data_reformatted[feature_discreet + '_2']
    
    cohen_kappas.at[counter, 'rater_A'] = 0
    cohen_kappas.at[counter, 'rater_B'] = 2
    cohen_kappas.at[counter, feature_discreet] = cohen_kappa_score(rater_0, rater_2)
    
    counter += 1
    
    rater_1 = data_reformatted[feature_discreet + '_1']
    rater_2 = data_reformatted[feature_discreet + '_2']
    
    cohen_kappas.at[counter, 'rater_A'] = 1
    cohen_kappas.at[counter, 'rater_B'] = 2
    cohen_kappas.at[counter, feature_discreet] = cohen_kappa_score(rater_1, rater_2)
    

counter = 0
data_subset = data_reformatted[['laughter_who_0', 'laughter_who_1']]
data_subset_no_nans = data_subset.dropna()

rater_0 = data_subset_no_nans['laughter_who_0']
rater_1 = data_subset_no_nans['laughter_who_1']

cohen_kappas.at[counter, 'rater_A'] = 0
cohen_kappas.at[counter, 'rater_B'] = 1
cohen_kappas.at[counter, 'laughter_who'] = cohen_kappa_score(rater_0, rater_1)

counter += 1

data_subset = data_reformatted[['laughter_who_0', 'laughter_who_2']]
data_subset_no_nans = data_subset.dropna()

rater_0 = data_subset_no_nans['laughter_who_0']
rater_2 = data_subset_no_nans['laughter_who_2']

cohen_kappas.at[counter, 'rater_A'] = 0
cohen_kappas.at[counter, 'rater_B'] = 2
cohen_kappas.at[counter, 'laughter_who'] = cohen_kappa_score(rater_0, rater_2)

counter += 1

data_subset = data_reformatted[['laughter_who_1', 'laughter_who_2']]
data_subset_no_nans = data_subset.dropna()

rater_1 = data_subset_no_nans['laughter_who_1']
rater_2 = data_subset_no_nans['laughter_who_2']

cohen_kappas.at[counter, 'rater_A'] = 1
cohen_kappas.at[counter, 'rater_B'] = 2
cohen_kappas.at[counter, 'laughter_who'] = cohen_kappa_score(rater_1, rater_2)

cohen_kappas
    

,rater_A,rater_B,laughter,gestures,laughter_who
0,0.0,1.0,0.832496,0.585153,0.530551
1,0.0,2.0,0.832496,0.262899,0.848485
2,1.0,2.0,0.869792,0.249758,0.464865


In [7]:
cohen_kappas[['laughter', 'laughter_who', 'gestures']].mean(axis=0)

laughter        0.844928
laughter_who    0.614634
gestures        0.365937
dtype: float64

**Intraclass correlation coefficient**

interpretation of scores here: Koo, T. K., & Li, M. Y. (2016). A guideline of selecting and reporting intraclass correlation coefficients for reliability research. Journal of chiropractic medicine, 15(2), 155-163.

package: https://pingouin-stats.org/generated/pingouin.intraclass_corr.html

In [8]:
import pingouin as pg

pg.intraclass_corr(data=data_long, targets='video_num', raters='rater',
                         ratings='awkward')

/Users/emma/anaconda3/lib/python3.6/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/emma/anaconda3/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.673529,7.189176,99,200,3.538343e-32,"[0.58, 0.75]"
1,ICC2,Single random raters,0.676358,7.817065,99,198,1.856891e-34,"[0.58, 0.76]"
2,ICC3,Single fixed raters,0.694410,7.817065,99,198,1.856891e-34,"[0.61, 0.77]"
3,ICC1k,Average raters absolute,0.860902,7.189176,99,200,3.538343e-32,"[0.81, 0.9]"
4,ICC2k,Average random raters,0.862439,7.817065,99,198,1.856891e-34,"[0.8, 0.9]"
5,ICC3k,Average fixed raters,0.872075,7.817065,99,198,1.856891e-34,"[0.82, 0.91]"


/Users/emma/anaconda3/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [9]:
pg.intraclass_corr(data=data_long, targets='video_num', raters='rater',
                         ratings='connected')

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.628025,6.065062,99,200,2.374485e-27,"[0.53, 0.72]"
1,ICC2,Single random raters,0.628885,6.180593,99,198,1.044925e-27,"[0.53, 0.72]"
2,ICC3,Single fixed raters,0.633278,6.180593,99,198,1.044925e-27,"[0.53, 0.72]"
3,ICC1k,Average raters absolute,0.835121,6.065062,99,200,2.374485e-27,"[0.77, 0.88]"
4,ICC2k,Average random raters,0.835628,6.180593,99,198,1.044925e-27,"[0.77, 0.88]"
5,ICC3k,Average fixed raters,0.838203,6.180593,99,198,1.044925e-27,"[0.77, 0.89]"


In [10]:
pg.intraclass_corr(data=data_long, targets='video_num', raters='rater',
                         ratings='topics')

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.811067,13.878683,99,200,1.437998e-53,"[0.75, 0.86]"
1,ICC2,Single random raters,0.811568,14.489281,99,198,1.046477e-54,"[0.75, 0.86]"
2,ICC3,Single fixed raters,0.818064,14.489281,99,198,1.046477e-54,"[0.76, 0.87]"
3,ICC1k,Average raters absolute,0.927947,13.878683,99,200,1.437998e-53,"[0.9, 0.95]"
4,ICC2k,Average random raters,0.928165,14.489281,99,198,1.046477e-54,"[0.9, 0.95]"
5,ICC3k,Average fixed raters,0.930983,14.489281,99,198,1.046477e-54,"[0.9, 0.95]"


In [11]:
pg.intraclass_corr(data=data_long, targets='video_num', raters='rater',
                         ratings='laughter_genuine', nan_policy='omit')

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.393668,2.947787,31,64,0.000130,"[0.18, 0.61]"
1,ICC2,Single random raters,0.419577,3.783362,31,62,0.000004,"[0.19, 0.63]"
2,ICC3,Single fixed raters,0.481271,3.783362,31,62,0.000004,"[0.27, 0.68]"
3,ICC1k,Average raters absolute,0.660762,2.947787,31,64,0.000130,"[0.39, 0.82]"
4,ICC2k,Average random raters,0.684407,3.783362,31,62,0.000004,"[0.42, 0.84]"
5,ICC3k,Average fixed raters,0.735685,3.783362,31,62,0.000004,"[0.52, 0.86]"


# Create average / consensus scores

In [12]:
features_continuous = ['awkward', 'connected', 'topics', 'laughter_genuine']

features_discreet = ['laughter', 'laughter_who', 'gestures']

for feature_continuous in features_continuous:
    
    column_list = [col for col in data_reformatted.columns if feature_continuous in col]
    new_column_list = []
    for column in column_list:
        rater = column.split(feature_continuous)[1].split('_')[1]
        if rater.isdigit():
            new_column_list.append(column)
            
    data_reformatted[feature_continuous + '_mean'] = data_reformatted[new_column_list].mean(axis=1)
        
for feature_discreet in features_discreet:
    
    column_list = [col for col in data_reformatted.columns if feature_discreet in col]

    new_column_list = []
    for column in column_list:
        rater = column.split(feature_discreet)[1].split('_')[1]
        if rater.isdigit():
            new_column_list.append(column)
            
    data_reformatted[feature_discreet + '_consensus'] = data_reformatted[new_column_list].mode(axis=1)[0]

for i in range(len(data_reformatted)):
    
    if data_reformatted['laughter_consensus'][i] == 2:
        data_reformatted.at[i,'laughter_who_consensus'] = np.nan

Turn laughter_who_consensus into a binary: alone / together

In [13]:
for i in range(len(data_reformatted)):
    
    if data_reformatted['laughter_who_consensus'][i] == 3:
        data_reformatted.at[i,'laughter_who_consensus_binary'] = 2
    elif data_reformatted['laughter_who_consensus'][i] in [1,2]:
        data_reformatted.at[i,'laughter_who_consensus_binary'] = 1
    else:
        data_reformatted.at[i,'laughter_who_consensus_binary'] = np.nan

In [14]:
data_reformatted.head()

,video_num,awkward_0,awkward_1,awkward_2,connected_0,connected_1,connected_2,topics_0,topics_1,topics_2,laughter_0,laughter_1,laughter_2,laughter_who_0,laughter_who_1,laughter_who_2,laughter_genuine_0,laughter_genuine_1,laughter_genuine_2,gestures_0,gestures_1,gestures_2,gestures_describe_0,gestures_describe_1,gestures_describe_2,rater_know_0,rater_know_1,rater_know_2,rater_know_how_0,rater_know_how_1,rater_know_how_2,notes_0,notes_1,notes_2,awkward_mean,connected_mean,topics_mean,laughter_genuine_mean,laughter_consensus,laughter_who_consensus,gestures_consensus,laughter_who_consensus_binary
0,1,68.0,20.0,61.0,14.0,63.0,55.0,50.0,24.0,68.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,nan,Each person nodded slightly,nan,1.0,1.0,1.0,Both are in the same sorority as me; the perso...,The person on the left was my freshman trip le...,friends,nan,nan,nan,49.666667,44.000000,47.333333,NaN,2.0,NaN,2.0,NaN
1,2,0.0,16.0,17.0,0.0,72.0,45.0,80.0,13.0,29.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,nan,nan,nan,2.0,1.0,2.0,NaN,The person on the left is a friend,NaN,nan,nan,nan,11.000000,39.000000,40.666667,NaN,2.0,NaN,2.0,NaN
2,3,9.0,6.0,2.0,70.0,88.0,71.0,94.0,78.0,94.0,1.0,1.0,1.0,3.0,3.0,3.0,6.0,7.0,7.0,2.0,2.0,2.0,nan,nan,nan,2.0,2.0,1.0,NaN,NaN,classmate,nan,nan,nan,5.666667,76.333333,88.666667,6.666667,1.0,3.0,2.0,2.0
3,4,13.0,68.0,55.0,70.0,29.0,41.0,78.0,71.0,80.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,Person on the left nodding his head,The person on the left nodded their head a lot...,nan,2.0,2.0,2.0,NaN,NaN,NaN,nan,I think that they were talking about skiing at...,nan,45.333333,46.666667,76.333333,NaN,2.0,NaN,1.0,NaN
4,5,10.0,1.0,22.0,57.0,66.0,61.0,100.0,100.0,92.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,nan,The person on the right scrunched their face a...,squeezing eyes,2.0,2.0,2.0,NaN,NaN,NaN,nan,nan,nan,11.000000,61.333333,97.333333,NaN,2.0,NaN,1.0,NaN


# Export reformatted data

In [15]:
data_reformatted.to_csv(os.path.join(base_dir, 'Analyses', 'long_gap_ratings_wide_format.csv'), encoding='utf-8', index=False)


In [16]:
data_reformatted = pd.DataFrame()
data_reformatted = data_reformatted.fillna(0)
counter = 0

file = os.path.join(base_dir, 'Data', 'long_gap_ratings.csv')
data = pd.read_csv(file)

features = ['awkward', 'connected', 'topics', 'laughter', 'laughter_who', 'laughter_genuine', 
            'gestures', 'gestures_describe', 'rater_know', 'rater_know_how', 'notes']
    
for feature in features:
    
    for row in range(len(data)):

        column_list = [col for col in data.columns if feature in col]

        for column in column_list:

            video = column.split(feature)[1].split('_')[1]
            
            if video.isdigit():

                data_reformatted.at[counter, 'video_num'] = video

                value = data[column][row]

                if feature in ['gestures_describe', 'rater_know_how', 'notes']:
                    data_reformatted.at[counter, feature + '_' + str(row)] = str(value)
                else:
                    data_reformatted.at[counter, feature + '_' + str(row)] = value

                counter += 1
            
        counter = 0
            
for i in range(3):
    data_reformatted.rename(columns = {'laughter_'+str(i): 'laughter_present_'+str(i),
                                      'gestures_'+str(i): 'gestures_present_'+str(i),
                                      'rater_know_'+str(i): 'rater_know_person_'+str(i)}, inplace = True)
    

In [17]:
data_long = pd.wide_to_long(data_reformatted, ['awkward', 'connected',
                                               'topics', 'laughter_genuine',
                                               'laughter_present','laughter_who', 'gestures_present',
                                               'gestures_describe', 'rater_know_person',
                                               'rater_know_how',
                                               'notes',],
                                    i='video_num', j='rater', sep='_', suffix='\w+')

data_long = data_long.reset_index()

In [18]:
data_long.to_csv(os.path.join(base_dir, 'Analyses', 'long_gap_ratings_long_format.csv'), encoding='utf-8', index=False)
